In [202]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.utils.data import TensorDataset # 텐서데이터셋
from torch.utils.data import DataLoader 

In [203]:
train = pd.read_csv('C:/Users/oyoungsun/train.csv')
test = pd.read_csv('C:/Users/oyoungsun/test.csv')

In [204]:
train.head(1)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NaN,S


In [205]:
test.head(1)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q


In [206]:
train.isnull().sum() # 비어있는 값이 있는지 개수 -> 결측치 체크

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [207]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [208]:
train = train.drop(['Cabin', 'Embarked', 'Name', 'Ticket', 'PassengerId'],axis=1)
test = test.drop(['Cabin', 'Embarked', 'Name', 'Ticket',PassengerId],axis=1)

NameError: name 'PassengerId' is not defined

In [209]:
train["Age"].fillna(train.groupby("Sex")["Age"].transform("mean"),inplace = True)
test["Age"].fillna(test.groupby("Sex")["Age"].transform("mean"), inplace=True)
test["Fare"].fillna(test.groupby("Sex")["Fare"].transform("median"), inplace=True)

In [210]:
train.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
dtype: int64

In [211]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
dtype: int64

In [212]:
#문자열을 숫자로 바꿔준다.

sex_mapping = {"male": 0, "female": 1}
train['Sex'] = train['Sex'].map(sex_mapping)
test['Sex'] = test['Sex'].map(sex_mapping)

In [213]:
train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,0,22.0,1,0,7.2500
1,1,1,1,38.0,1,0,71.2833
2,1,3,1,26.0,0,0,7.9250
3,1,1,1,35.0,1,0,53.1000
4,0,3,0,35.0,0,0,8.0500


In [232]:
# 나이, fare 이상치 안 지우고 진행할 것임. ->  0.673657
#이상치 지워보자 -> Cost: 0.555345? 
#그런데 배치사이즈가 바뀌어서 그런 걸 수도 있음.
age_mean = train['Age'].mean()
age_std = train['Age'].std()
indexNames = train[train['Age'] < age_mean - 3*age_std].index
train.drop(indexNames , inplace=True)
indexNames = train[train['Age'] > age_mean + 3*age_std].index
train.drop(indexNames , inplace=True)
fare_mean = train['Fare'].mean()
fare_std = train['Fare'].std()
indexNames = train[train['Fare'] < fare_mean - 3*fare_std].index
train.drop(indexNames , inplace=True)
indexNames = train[train['Fare'] > fare_mean + 3*fare_std].index
train.drop(indexNames , inplace=True)

In [233]:
x_train = torch.Tensor([train['Pclass'], train['Sex'], train['Age'], train['SibSp'], train['Parch'], train['Fare']])
x_train = x_train.transpose(0,1)
print(x_train[0])
y_train = torch.Tensor(train['Survived'])
y_train = y_train.type(torch.LongTensor)
x_train.shape

tensor([ 3.0000,  0.0000, 22.0000,  1.0000,  0.0000,  7.2500])


torch.Size([864, 6])

In [237]:
dataset = TensorDataset(x_train, y_train)
dataloader = DataLoader(dataset, batch_size=100, shuffle=True, drop_last=True)

In [238]:
W = torch.zeros(6,1,requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W,b], lr=0.0005)
nb_epochs = 100
for epoch in range(nb_epochs + 1):
    for batch_idx, samples in enumerate(dataloader) :
        x_train, y_train = samples
        hypothesis = torch.sigmoid(x_train.matmul(W) + b)
      #  print(hypothesis)
        cost = -(y_train * torch.log(hypothesis) + (1 - y_train) * torch.log(1 - hypothesis)).mean()
    #    print(cost)
        # cost로 H(x) 누적 개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        
        #100번마다 로그 출력
    if epoch % 10== 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format( epoch, nb_epochs, cost.item()))

Epoch    0/100 Cost: 0.555345
Epoch   10/100 Cost: 0.555345
Epoch   20/100 Cost: 0.555345
Epoch   30/100 Cost: 0.555345
Epoch   40/100 Cost: 0.555345
Epoch   50/100 Cost: 0.555345
Epoch   60/100 Cost: 0.555345
Epoch   70/100 Cost: 0.555345
Epoch   80/100 Cost: 0.555345
Epoch   90/100 Cost: 0.555345
Epoch  100/100 Cost: 0.555345


In [ ]:
'''training_epochs = 15
batch_size = 100

linear = nn.Linear(6, 1, bias = True)


criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(linear.parameters(), lr=0.1)
nb_epochs = 10000
for epoch in range(nb_epochs + 1):
    optimizer.zero_grad()
    
    hypothesis = linear(x_train)
    cost = criterion(hypothesis, y_train)
    # cost로 H(x) 누적 개선

    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

In [244]:
test.head(200)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",0,34.500000,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",1,47.000000,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",0,62.000000,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",0,27.000000,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,22.000000,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
195,1087,3,"Karlsson, Mr. Julius Konrad Eugen",0,33.000000,0,0,347465,7.8542,NaN,S
196,1088,1,"Spedden, Master. Robert Douglas",0,6.000000,0,2,16966,134.5000,E34,C
197,1089,3,"Nilsson, Miss. Berta Olivia",1,18.000000,0,0,347066,7.7750,NaN,S
198,1090,2,"Baimbrigge, Mr. Charles Robert",0,23.000000,0,0,C.A. 31030,10.5000,NaN,S


In [263]:
test.drop(['PassengerId', 'Age', 'Parch', 'Fare', 'SibSp'], axis=1)
x_test = torch.Tensor([test['Pclass'], test['Sex'], test['Age'], test['SibSp'], test['Parch'],test['Fare']])
x_test = x_test.transpose(0,1)

predict = linear(x_test).tolist()


In [264]:

#print(predict)
result =pd.DataFrame({
    'PassengerId': test['PassengerId'],   
    'Survived': predict
})
result.to_csv('result.csv', index=False) 